In [ ]:
import scipy.io as sio
import numpy as np
import sys
import matplotlib.pyplot as plt
sys.path.append('/Users/edwardmcdugald/Research/convection_patterns_wip/code/ml_experiments')
from myPDEFIND import *
#sys.path.append('/Users/edwardmcdugald/Research/convection_patterns_wip/code/numerics')
#import convection_patterns_v2 as cp

#cp.solveSH(20*np.pi,20*np.pi,256,256,.1,100,"SHAutoEncode_3",Rscale=.5,
#            beta=.45,amplitude=.1,init_flag=1,energy=False)
data = sio.loadmat("/Users/edwardmcdugald/Research/convection_patterns_wip/code/data/SHAutoEncode_3.mat")
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(3,3))
ax.imshow(data['uu'][500,:,:])


In [ ]:
tt = data['tt'][0][1:] #TVALUES, EXCLUDING FIRST ONE
xx = data['xx']
yy = data['yy']
U = data['uu'][1:1001,:,:] #TAKE LAST 1000 U surfaces
dt = data['tt'][0][1]-data['tt'][0][0]
Ut = BackwardDiff(U,data['uu'][0:1000,:,:],dt) #Compute U derivatives
n_samples = len(tt)
N = len(xx)*len(yy)
perm = np.random.permutation(int(.9*n_samples))
training_samples = perm[:int(.8*n_samples)]
val_samples = perm[int(.8*n_samples):]
test_samples = np.arange(int(.9*n_samples), n_samples)

#subsampling by factor of 2
training_data = {'tt': tt[training_samples],
                     'xx': xx[::2].T,
                     'yy': yy[::2].T,
                     'U': U[training_samples,::2,::2],
                     'Ut': Ut[training_samples,::2,::2]}
val_data = {'tt': tt[val_samples],
                'xx': xx[::2].T,
                'yy': yy[::2].T,
                'U': U[val_samples,::2,::2],
                'Ut': Ut[val_samples,::2,::2]}
test_data = {'tt': tt[test_samples],
                 'xx': xx[::2].T,
                 'yy': yy[::2].T,
                 'U': U[test_samples,::2,::2],
                 'Ut': Ut[test_samples,::2,::2]}


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model


In [ ]:
u_train = training_data['U']
u_test = test_data['U']
u_val = val_data['U']
u_train = u_train.astype('float32')
u_test = u_test.astype('float32')
u_val = u_val.astype('float32')
print(u_train.shape)
print(u_test.shape)
print(u_val.shape)


In [ ]:
latent_dim = 1024

class Autoencoder(Model):
  def __init__(self, latent_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(256, activation='linear'),
      layers.Dense(512, activation='sigmoid'),
      layers.Dense(latent_dim, activation='relu'),
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(2048, activation='linear'),
      layers.Dense(4096, activation='sigmoid'),
      layers.Dense(8192, activation='relu'),
      layers.Dense(16384, activation='linear'),
      layers.Reshape((128, 128))
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = Autoencoder(latent_dim)
autoencoder.encoder.summary()
autoencoder.decoder.summary()
autoencoder.summary()


In [ ]:
def my_loss_fn(x_in, x_out, dt):
    squared_difference = tf.square(x_in - x_out)
    ae_err = tf.reduce_mean(squared_difference, axis=-1)
    z = autoencoder.encoder(x_in).numpy().reshape(32,32)
    z_derivs = BackwardDiff(z[1:len(z)],z[0:len(z)-1],dt)

    return tf.reduce_mean(squared_difference, axis=-1)  # Note the `axis=-1`

In [ ]:

autoencoder.compile(optimizer='adam', loss=my_loss_fn)
autoencoder.fit(u_train, u_train,
                epochs=10,
                batch_size=5,
                shuffle=True,
                validation_data=(u_test, u_test))


In [ ]:
encoded_data = autoencoder.encoder(u_val).numpy()
decoded_data = autoencoder.decoder(encoded_data).numpy()


In [ ]:
n = 10
plt.figure(figsize=(20, 6))
for i in range(n):
  idx = np.random.randint(1,99)
  # display original
  ax = plt.subplot(3, n, i + 1)
  plt.imshow(u_val[idx])
  plt.title("original"+"_"+str(idx))
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

  # display reshaped encoded
  ax = plt.subplot(3, n, i + 1 + n)
  plt.imshow(encoded_data[idx].reshape(32,32))
  plt.title("encoded"+"_"+str(idx))
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

  # display reconstruction
  ax = plt.subplot(3, n, i + 1 + 2*n)
  plt.imshow(decoded_data[idx])
  plt.title("reconstructed"+"_"+str(idx))
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
plt.show()


In [ ]:
import math

t = data['tt'].T[:,0]
x = data['xx'].T[0,:]
y = data['yy'].T[0,:]
U = data['uu']
print("data shape: ",np.shape(U))

nx = len(x)
ny = len(y)
dx = x[1]-x[0]
dy = y[1]-y[0]
dt = t[1]-t[0]
print("dt=",dt,"dx=",dx,"dy=",dy)

num_t = 20
x_subsample = 8
y_subsample = 8
t_vals = np.arange(1,len(t),math.floor(len(t)/num_t))
x_vals = np.arange(0,nx,x_subsample)
y_vals = np.arange(0,ny,y_subsample)

num_points = num_t*len(x_vals)*len(y_vals)
print("feature vec length: ",num_points)
print(dt)

u = np.zeros((num_points,1))
u_t = np.zeros((num_points,1))
u_x = np.zeros((num_points,1))
u_y = np.zeros((num_points,1))
u_xx = np.zeros((num_points,1))
u_yy = np.zeros((num_points,1))
u_xy = np.zeros((num_points,1))
lapu = np.zeros((num_points,1))
biharmu = np.zeros((num_points,1))


# setting parameters for spectral derivatives
Lx = 2*x[len(x)-1] # Size of enclosing periodic rectangle
Ly = 2*y[len(y)-1]

i=0
for t_idx in t_vals:
    print(t_idx)
    uu = U[t_idx,:,:]
    uu_t = BackwardDiff(U[t_idx,:,:],U[t_idx-1,:,:],dt)
    uu_x = SpectralDerivs(U[t_idx,:,:],Lx,Ly,'x')
    uu_y = SpectralDerivs(U[t_idx,:,:],Lx,Ly,'y')
    uu_xx = SpectralDerivs(U[t_idx,:,:],Lx,Ly,'xx')
    uu_yy = SpectralDerivs(U[t_idx,:,:],Lx,Ly,'yy')
    uu_xy = SpectralDerivs(U[t_idx,:,:],Lx,Ly,'xy')
    lapuu = SpectralDerivs(U[t_idx,:,:],Lx,Ly,'laplacian')
    biharmuu = SpectralDerivs(U[t_idx,:,:],Lx,Ly,'biharmonic')
    for x_idx in x_vals:
        for y_idx in y_vals:
            u[i] = uu[x_idx,y_idx]
            u_t[i] = uu_t[x_idx,y_idx]
            u_x[i] = uu_x[x_idx,y_idx]
            u_y[i] = uu_y[x_idx,y_idx]
            u_xy[i] = uu_xy[x_idx,y_idx]
            lapu[i] = lapuu[x_idx,y_idx]
            biharmu[i] = biharmuu[x_idx,y_idx]
            i+=1

X = np.hstack([np.ones((num_points,1)), u, u**2, u**3,u**4,
                   u_x, u_y, u_x**2, u_y**2, u_x*u_y,u_xy,
               lapu, biharmu])
description = ['','u','u^2','u^3','u^4',
               'u_{x}','u_{y}','u_{x}^2','u_{y}^2','u_{x}u_{y}','u_{xy}'
    ,'lapu','biharmu']

c = TrainSTRidge(X,u_t,10**-5,1)
print_pde(c, description)